In [ ]:
# 비트코인, 이더리움, 위믹스 토큰의 가격 및 거래량의 일별 자료를 통해 비트코인 과의 상관관계를 알아보고자 함.
# 3개년 치 (2020-10-26 ~ 2022-12-26) 의 자료를 investing.com 의 과거 데이터 다운로드를 통해 자료 수집.
# 데이터 전처리 후 matplotlib를 통해 각종 도표로 데이터 확인.

In [2]:
import pandas as pd

In [3]:
# %load matplot.py
# %%
import matplotlib.pyplot as plt
plt.rc('font', family = 'gulim')
plt.rc('axes', unicode_minus=False)




In [4]:
bit_origin = pd.read_csv('./data/bit.csv')
eth_origin = pd.read_csv('./data/eth.csv')
wemix_origin = pd.read_csv('./data/wemix.csv')

# 각 데이터 프레임을 copy에 할당함
bit_copy, eth_copy, wemix_copy = bit_origin.copy(), eth_origin.copy(), wemix_origin.copy()
print(bit_copy.head())

              날짜        종가        오픈        고가        저가      거래량    변동 %
0  2022년 12월 26일  16,831.9  16,831.8  16,836.5  16,822.0  133.33K   0.00%
1  2022년 12월 25일  16,831.8  16,835.7  16,852.9  16,733.0  132.89K  -0.03%
2  2022년 12월 24일  16,837.2  16,778.6  16,855.8  16,777.8  110.12K   0.35%
3  2022년 12월 23일  16,779.1  16,820.5  16,910.7  16,768.0  184.12K  -0.25%
4  2022년 12월 22일  16,820.6  16,831.8  16,862.2  16,566.1  198.28K  -0.07%


### bit_copy 데이터 전처리

In [5]:
bit_copy.isnull().sum() # 다행히 NaN 값은 없음.
bit_copy.info() # 모든 컬럼의 데이터 타입이 object로 되어있음.
# 종가, 오픈, 고가, 저가의 컬럼 데이터 타입을 float로 변경하고자 함.
bit_copy['종가'] = bit_copy['종가'].astype('float')
# string 데이터에 comma(,)가 있기 때문에 바로 전환이 안됨.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      761 non-null    object
 1   종가      761 non-null    object
 2   오픈      761 non-null    object
 3   고가      761 non-null    object
 4   저가      761 non-null    object
 5   거래량     761 non-null    object
 6   변동 %    761 non-null    object
dtypes: object(7)
memory usage: 41.7+ KB


ValueError: could not convert string to float: '16,831.9'

In [6]:
# Python: Convert string with comma separator and dot to float
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

bit_copy['종가'] = bit_copy['종가'].map(locale.atof)

In [7]:
bit_copy['종가'].dtype # '종가'컬럼의 데이터 타입이 float64로 변경됨

dtype('float64')

In [8]:
# bit_copy[['오픈','고가','저가']] = bit_copy[['오픈','고가','저가']].map(locale.atof)
# 위처럼 하니 오류가 발생하여 그냥 하나씩 진행함
bit_copy['오픈'] = bit_copy['오픈'].map(locale.atof)
bit_copy['고가'] = bit_copy['고가'].map(locale.atof)
bit_copy['저가'] = bit_copy['저가'].map(locale.atof)

In [9]:
bit_copy.info() # 오픈, 고가, 저가의 데이터 타입이 float64로 변경된 것을 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      761 non-null    object 
 1   종가      761 non-null    float64
 2   오픈      761 non-null    float64
 3   고가      761 non-null    float64
 4   저가      761 non-null    float64
 5   거래량     761 non-null    object 
 6   변동 %    761 non-null    object 
dtypes: float64(4), object(3)
memory usage: 41.7+ KB


In [10]:
bit_copy['거래량'].head() # 거래량의 K는 1000을 뜻함. 이를 K를 없애고 일반 숫자로 변환하고 싶음.

0    133.33K
1    132.89K
2    110.12K
3    184.12K
4    198.28K
Name: 거래량, dtype: object

In [13]:
for i in range(len(bit_copy['거래량'])):
    bit_copy['거래량'][i] = bit_copy['거래량'][i][:-1] # 마지막 단어 K를 없앰
bit_copy['거래량'].head()

C:\Users\Jasper\AppData\Local\Temp\ipykernel_32912\1011294417.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_copy['거래량'][i] = bit_copy['거래량'][i][:-1] # 마지막 단어 K를 없앰


0    133.33
1    132.89
2    110.12
3    184.12
4    198.28
Name: 거래량, dtype: object

In [14]:
bit_copy['거래량'] = bit_copy['거래량'].map(locale.atof) #거래량을 float64로 변경
bit_copy['거래량'] *= 1000 # 단위수 맞춤

In [15]:
bit_copy.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2022년 12월 26일,16831.9,16831.8,16836.5,16822.0,133330.0,0.00%
1,2022년 12월 25일,16831.8,16835.7,16852.9,16733.0,132890.0,-0.03%
2,2022년 12월 24일,16837.2,16778.6,16855.8,16777.8,110120.0,0.35%
3,2022년 12월 23일,16779.1,16820.5,16910.7,16768.0,184120.0,-0.25%
4,2022년 12월 22일,16820.6,16831.8,16862.2,16566.1,198280.0,-0.07%


In [16]:
# 날짜를 연, 월, 일로 분리하고 싶음
bit_copy['날짜'].head()

0    2022년 12월 26일
1    2022년 12월 25일
2    2022년 12월 24일
3    2022년 12월 23일
4    2022년 12월 22일
Name: 날짜, dtype: object

In [17]:
# bit_copy_date = bit_copy.copy()
# 날짜의 데이터는 '2022년 12월 26일'로 구분자가 3개가 있음.
# 구분자를 하나로 변경.
bit_copy['날짜'] = bit_copy['날짜'].apply(lambda x: x.replace('년 ','-').replace('월 ', '-').replace('일', ''))

dates = bit_copy['날짜'].str.split('-') # 리스트로 데이터 리턴
bit_copy['연'] = dates.str.get(0)
bit_copy['월'] = dates.str.get(1)
bit_copy['일'] = dates.str.get(2)

In [18]:
bit_copy.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %,연,월,일
0,2022-12-26,16831.9,16831.8,16836.5,16822.0,133330.0,0.00%,2022,12,26
1,2022-12-25,16831.8,16835.7,16852.9,16733.0,132890.0,-0.03%,2022,12,25
2,2022-12-24,16837.2,16778.6,16855.8,16777.8,110120.0,0.35%,2022,12,24
3,2022-12-23,16779.1,16820.5,16910.7,16768.0,184120.0,-0.25%,2022,12,23
4,2022-12-22,16820.6,16831.8,16862.2,16566.1,198280.0,-0.07%,2022,12,22


In [19]:
# '변동 %'를 '증감율'로 컬럼명 변경 -> 추후 drop 해도 될듯
bit_copy.rename(columns={'변동 %':'증감율'}, inplace=True)

In [20]:
# 증감율을 계산에 활용할 수 있게끔 float로 변경
for i in range(len(bit_copy['증감율'])):
    bit_copy['증감율'][i] = bit_copy['증감율'][i][:-1] # 마지막 단어 %를 없앰
bit_copy['증감율'] = bit_copy['증감율'].map(locale.atof) #거래량을 float64로 변경
bit_copy['증감율'] /= 100 # 퍼센트 단위수 맞춤
bit_copy['증감율'].head()

C:\Users\Jasper\AppData\Local\Temp\ipykernel_32912\3854986562.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_copy['증감율'][i] = bit_copy['증감율'][i][:-1] # 마지막 단어 %를 없앰


0    0.0000
1   -0.0003
2    0.0035
3   -0.0025
4   -0.0007
Name: 증감율, dtype: float64

In [21]:
# '오픈'을 '시가'로 컬럼명 변경
bit_copy.rename(columns={'오픈':'시가'}, inplace=True)

In [22]:
# '종가'를 '당일종가'로 컬럼명 변경
bit_copy.rename(columns={'종가':'당일종가'}, inplace=True)

In [23]:
# '전일종가' 컬럼을 추가, 증감율 계산 확인을 위해..?
bit_copy = bit_copy.copy()
bit_copy['전일종가'] = bit_copy.loc[:,'당일종가'].shift(-1)
bit_copy.tail() # 마지막 열의 전일 종가가 없음. 증감율을 확인하여 임의로 대입
bit_copy['전일종가'].fillna(method='ffill', inplace=True)

In [24]:
# '종류'라는 컬럼에 '비트코인' 추가
bit_copy['종류'] = '비트코인'

In [25]:
# 데이터프레임 컬럼 순서 변경
bit_copy = bit_copy[['종류','날짜','연','월','일','당일종가','전일종가','증감율','시가','고가','저가','거래량']]
print(bit_copy.head())
print(bit_copy.tail())

     종류          날짜     연   월   일     당일종가     전일종가     증감율       시가       고가  \
0  비트코인  2022-12-26  2022  12  26  16831.9  16831.8  0.0000  16831.8  16836.5   
1  비트코인  2022-12-25  2022  12  25  16831.8  16837.2 -0.0003  16835.7  16852.9   
2  비트코인  2022-12-24  2022  12  24  16837.2  16779.1  0.0035  16778.6  16855.8   
3  비트코인  2022-12-23  2022  12  23  16779.1  16820.6 -0.0025  16820.5  16910.7   
4  비트코인  2022-12-22  2022  12  22  16820.6  16831.8 -0.0007  16831.8  16862.2   

        저가       거래량  
0  16822.0  133330.0  
1  16733.0  132890.0  
2  16777.8  110120.0  
3  16768.0  184120.0  
4  16566.1  198280.0  
       종류          날짜     연   월   일     당일종가     전일종가     증감율       시가  \
756  비트코인  2020-11-30  2020  11  30  19698.1  18185.5  0.0832  18186.0   
757  비트코인  2020-11-29  2020  11  29  18185.5  17730.7  0.0257  17729.7   
758  비트코인  2020-11-28  2020  11  28  17730.7  17127.1  0.0352  17142.6   
759  비트코인  2020-11-27  2020  11  27  17127.1  17162.0 -0.0020  17160.9   
760  

In [26]:
# 하다보니 현재 금액이 USD로 되어 있기 때문에 KRW로 변환하고 싶음
ex_rate = pd.read_excel('./data/MonAvgStdExRate.xlsx') # 월평균 매매 기준율(2020.10 ~ 2022.12) 생성 (서울외국환중개)
ex_rate.head()

,날짜,통화명,월평균 매매기준율
0,2020.10,미국 달러 (USD),1144.68
1,2020.11,미국 달러 (USD),1116.76
2,2020.12,미국 달러 (USD),1095.13
3,2021.01,미국 달러 (USD),1097.49
4,2021.02,미국 달러 (USD),1111.72


In [27]:
print(ex_rate.tail(),end='\n') # 2022년 12월 환율이 없음. 임의로 행 추가
ex_rate.loc[26] = ['2022.12','',float(1302.02)] # 12월 매매기준율 평균
ex_rate.tail()

          날짜          통화명  월평균 매매기준율
21  2022.07　  미국 달러 (USD)    1307.40
22  2022.08　  미국 달러 (USD)    1318.44
23  2022.09　  미국 달러 (USD)    1391.59
24  2022.10　  미국 달러 (USD)    1426.66
25  2022.11　  미국 달러 (USD)    1364.10


,날짜,통화명,월평균 매매기준율
22,2022.08,미국 달러 (USD),1318.44
23,2022.09,미국 달러 (USD),1391.59
24,2022.10,미국 달러 (USD),1426.66
25,2022.11,미국 달러 (USD),1364.10
26,2022.12,,1302.02


In [28]:
# 날짜 기준 내림차순(최신순)으로 정렬
ex_rate.sort_values('날짜', ascending=False, inplace=True)
print(ex_rate.head())

          날짜          통화명  월평균 매매기준율
26   2022.12                 1302.02
25  2022.11　  미국 달러 (USD)    1364.10
24  2022.10　  미국 달러 (USD)    1426.66
23  2022.09　  미국 달러 (USD)    1391.59
22  2022.08　  미국 달러 (USD)    1318.44


In [29]:
# 날짜를 연 월로 분리 (2020.12 형식임)
ex_dates = ex_rate['날짜'].str.split('.') # 리스트로 데이터 리턴
ex_rate['연'] = ex_dates.str.get(0)
ex_rate['월'] = ex_dates.str.get(1)
ex_rate.head()

,날짜,통화명,월평균 매매기준율,연,월
26,2022.12,,1302.02,2022,12
25,2022.11,미국 달러 (USD),1364.10,2022,11
24,2022.10,미국 달러 (USD),1426.66,2022,10
23,2022.09,미국 달러 (USD),1391.59,2022,09
22,2022.08,미국 달러 (USD),1318.44,2022,08


In [30]:
# 날짜, 통화명 제거
del ex_rate['날짜']
del ex_rate['통화명']

In [31]:
# 컬럼명 변경
ex_rate.rename(columns={'월평균 매매기준율':'환율'}, inplace=True)

In [32]:
# 컬럼 순서 변경
ex_rate = ex_rate[['연','월','환율']]
print(ex_rate.head())

# '월' 컬럼에 공백이 있음으로 처리
ex_rate['월'] = ex_rate['월'].str.strip()
print(ex_rate.head())

       연    월       환율
26  2022   12  1302.02
25  2022  11　  1364.10
24  2022  10　  1426.66
23  2022  09　  1391.59
22  2022  08　  1318.44
       연   월       환율
26  2022  12  1302.02
25  2022  11  1364.10
24  2022  10  1426.66
23  2022  09  1391.59
22  2022  08  1318.44


In [33]:
# 연 월로 환율 조회하는 방법
ex_rate.loc[(ex_rate['연'] =='2020') & (ex_rate['월'] =='11'),['환율']]

,환율
1,1116.76


In [34]:
# bit_krw df 새로 생성
bit_krw = bit_copy.copy()
# 원달러환율 컬럼 추가, ex_rate의 연월 일치하는 환율을 찾아서 대입
bit_krw['원달러환율'] = 0

# 계산을 위한 초기 변수 설정
years = ['2020','2021','2022']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
krw_rate = 0

for year in years:
    for month in months:
        krw_rate = ex_rate.loc[(ex_rate['연'] ==year) & (ex_rate['월'] ==month),['환율']]
        krw_rate = krw_rate.squeeze() #데이터프레임을 시리즈로 변환하여 계산할수 있게 함.
        bit_krw.loc[(bit_krw['연'] ==year) & (bit_krw['월'] ==month),['원달러환율']] = krw_rate
print(bit_krw.head())
print(bit_krw.tail())

     종류          날짜     연   월   일     당일종가     전일종가     증감율       시가       고가  \
0  비트코인  2022-12-26  2022  12  26  16831.9  16831.8  0.0000  16831.8  16836.5   
1  비트코인  2022-12-25  2022  12  25  16831.8  16837.2 -0.0003  16835.7  16852.9   
2  비트코인  2022-12-24  2022  12  24  16837.2  16779.1  0.0035  16778.6  16855.8   
3  비트코인  2022-12-23  2022  12  23  16779.1  16820.6 -0.0025  16820.5  16910.7   
4  비트코인  2022-12-22  2022  12  22  16820.6  16831.8 -0.0007  16831.8  16862.2   

        저가       거래량    원달러환율  
0  16822.0  133330.0  1302.02  
1  16733.0  132890.0  1302.02  
2  16777.8  110120.0  1302.02  
3  16768.0  184120.0  1302.02  
4  16566.1  198280.0  1302.02  
       종류          날짜     연   월   일     당일종가     전일종가     증감율       시가  \
756  비트코인  2020-11-30  2020  11  30  19698.1  18185.5  0.0832  18186.0   
757  비트코인  2020-11-29  2020  11  29  18185.5  17730.7  0.0257  17729.7   
758  비트코인  2020-11-28  2020  11  28  17730.7  17127.1  0.0352  17142.6   
759  비트코인  2020-11-27  20

In [35]:
# 당일종가(KRW) 컬럼 추가
bit_krw['당일종가(KRW)'] = bit_krw['당일종가'] * bit_krw['원달러환율']

In [36]:
bit_krw.head()

,종류,날짜,연,월,일,당일종가,전일종가,증감율,시가,고가,저가,거래량,원달러환율,당일종가(KRW)
0,비트코인,2022-12-26,2022,12,26,16831.9,16831.8,0.0000,16831.8,16836.5,16822.0,133330.0,1302.02,2.191547e+07
1,비트코인,2022-12-25,2022,12,25,16831.8,16837.2,-0.0003,16835.7,16852.9,16733.0,132890.0,1302.02,2.191534e+07
2,비트코인,2022-12-24,2022,12,24,16837.2,16779.1,0.0035,16778.6,16855.8,16777.8,110120.0,1302.02,2.192237e+07
3,비트코인,2022-12-23,2022,12,23,16779.1,16820.6,-0.0025,16820.5,16910.7,16768.0,184120.0,1302.02,2.184672e+07
4,비트코인,2022-12-22,2022,12,22,16820.6,16831.8,-0.0007,16831.8,16862.2,16566.1,198280.0,1302.02,2.190076e+07


In [37]:
# 시가, 고가, 저가 삭제
del bit_krw['시가']
del bit_krw['고가']
del bit_krw['저가']

In [38]:
# 해보니 전일종가, 증감율도 필요 없어보임
del bit_krw['전일종가']
del bit_krw['증감율']

In [39]:
# 컬럼명 변경 및 컬럼 순서 변경
bit_krw.rename(columns={'당일종가':'종가(USD)'}, inplace=True)
bit_krw.rename(columns={'당일종가(KRW)':'종가(KRW)'}, inplace=True)

bit_krw = bit_krw[['종류','날짜','연','월','일','종가(USD)','종가(KRW)','원달러환율', '거래량']]

In [40]:
pd.set_option('display.float_format', '{:.2f}'.format) # 당일종가(KRW) 컬럼이 e로 나오는 것을 식별가능한 숫자로 변경
bit_krw.head()

# bit_krw 데이터 전처리. 끝.

,종류,날짜,연,월,일,종가(USD),종가(KRW),원달러환율,거래량
0,비트코인,2022-12-26,2022,12,26,16831.90,21915470.44,1302.02,133330.00
1,비트코인,2022-12-25,2022,12,25,16831.80,21915340.24,1302.02,132890.00
2,비트코인,2022-12-24,2022,12,24,16837.20,21922371.14,1302.02,110120.00
3,비트코인,2022-12-23,2022,12,23,16779.10,21846723.78,1302.02,184120.00
4,비트코인,2022-12-22,2022,12,22,16820.60,21900757.61,1302.02,198280.00


### eth_copy 데이터 전처리

In [41]:
# bit_copy에서 진행했던 절차를 따라 진행하되 필요 없는 절차는 하지 않는다.
eth_copy['종가'] = eth_copy['종가'].map(locale.atof)

In [42]:
eth_copy.head()
# 오픈, 고가, 저가, 변동 % 삭제
eth_copy.drop(eth_copy.columns[[2,3,4,6]], axis=1, inplace=True)

In [43]:
eth_copy.head()

,날짜,종가,거래량
0,2022년 12월 26일,1218.49,258.53K
1,2022년 12월 25일,1218.69,256.52K
2,2022년 12월 24일,1220.46,131.51K
3,2022년 12월 23일,1220.03,301.17K
4,2022년 12월 22일,1217.57,383.63K


In [44]:
# 거래량의 K는 1000을 뜻함.
# 1) K를 없애고 
for i in range(len(eth_copy['거래량'])):
    eth_copy['거래량'][i] = eth_copy['거래량'][i][:-1] # 마지막 단어 K를 없앰

# 2) 일반 숫자로 변환.
eth_copy['거래량'] = eth_copy['거래량'].map(locale.atof) #거래량을 float64로 변경
eth_copy['거래량'] *= 1000 # 단위수 맞춤

eth_copy.head()

C:\Users\Jasper\AppData\Local\Temp\ipykernel_32912\3698492884.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_copy['거래량'][i] = eth_copy['거래량'][i][:-1] # 마지막 단어 K를 없앰


,날짜,종가,거래량
0,2022년 12월 26일,1218.49,258530.00
1,2022년 12월 25일,1218.69,256520.00
2,2022년 12월 24일,1220.46,131510.00
3,2022년 12월 23일,1220.03,301170.00
4,2022년 12월 22일,1217.57,383630.00


In [45]:
# 날짜를 연, 월, 일로 분리하고 싶음

# 날짜의 데이터는 '2022년 12월 26일'로 구분자가 3개가 있음.
# 구분자를 하나로 변경.
eth_copy['날짜'] = eth_copy['날짜'].apply(lambda x: x.replace('년 ','-').replace('월 ', '-').replace('일', ''))

dates = eth_copy['날짜'].str.split('-') # 리스트로 데이터 리턴
eth_copy['연'] = dates.str.get(0)
eth_copy['월'] = dates.str.get(1)
eth_copy['일'] = dates.str.get(2)

eth_copy.head()

,날짜,종가,거래량,연,월,일
0,2022-12-26,1218.49,258530.00,2022,12,26
1,2022-12-25,1218.69,256520.00,2022,12,25
2,2022-12-24,1220.46,131510.00,2022,12,24
3,2022-12-23,1220.03,301170.00,2022,12,23
4,2022-12-22,1217.57,383630.00,2022,12,22


In [46]:
# '종류'라는 컬럼에 '이더리움' 추가
eth_copy['종류'] = '이더리움'
eth_copy.head()

,날짜,종가,거래량,연,월,일,종류
0,2022-12-26,1218.49,258530.00,2022,12,26,이더리움
1,2022-12-25,1218.69,256520.00,2022,12,25,이더리움
2,2022-12-24,1220.46,131510.00,2022,12,24,이더리움
3,2022-12-23,1220.03,301170.00,2022,12,23,이더리움
4,2022-12-22,1217.57,383630.00,2022,12,22,이더리움


In [47]:
# 하다보니 현재 금액이 USD로 되어 있기 때문에 KRW로 변환하고 싶음
# 기 처리된 ex_rate 데이터프레임 사용

# eth_krw df 새로 생성
eth_krw = eth_copy.copy()
# 원달러환율 컬럼 추가, ex_rate의 연월 일치하는 환율을 찾아서 대입
eth_krw['원달러환율'] = 0

# 계산을 위한 초기 변수 설정
years = ['2020','2021','2022']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
krw_rate = 0

for year in years:
    for month in months:
        krw_rate = ex_rate.loc[(ex_rate['연'] ==year) & (ex_rate['월'] ==month),['환율']]
        krw_rate = krw_rate.squeeze() #데이터프레임을 시리즈로 변환하여 계산할수 있게 함.
        eth_krw.loc[(eth_krw['연'] ==year) & (eth_krw['월'] ==month),['원달러환율']] = krw_rate
print(eth_krw.head())
print(eth_krw.tail())

           날짜      종가       거래량     연   월   일    종류   원달러환율
0  2022-12-26 1218.49 258530.00  2022  12  26  이더리움 1302.02
1  2022-12-25 1218.69 256520.00  2022  12  25  이더리움 1302.02
2  2022-12-24 1220.46 131510.00  2022  12  24  이더리움 1302.02
3  2022-12-23 1220.03 301170.00  2022  12  23  이더리움 1302.02
4  2022-12-22 1217.57 383630.00  2022  12  22  이더리움 1302.02
             날짜     종가      거래량     연   월   일    종류   원달러환율
756  2020-11-30 615.92 51720.00  2020  11  30  이더리움 1116.76
757  2020-11-29 575.45  5790.00  2020  11  29  이더리움 1116.76
758  2020-11-28 537.74  5340.00  2020  11  28  이더리움 1116.76
759  2020-11-27 518.11  6420.00  2020  11  27  이더리움 1116.76
760  2020-11-26 519.32 13110.00  2020  11  26  이더리움 1116.76


In [48]:
# 종가(KRW) 컬럼 추가
eth_krw['종가(KRW)'] = eth_krw['종가'] * eth_krw['원달러환율']
eth_krw.head()

,날짜,종가,거래량,연,월,일,종류,원달러환율,종가(KRW)
0,2022-12-26,1218.49,258530.00,2022,12,26,이더리움,1302.02,1586498.35
1,2022-12-25,1218.69,256520.00,2022,12,25,이더리움,1302.02,1586758.75
2,2022-12-24,1220.46,131510.00,2022,12,24,이더리움,1302.02,1589063.33
3,2022-12-23,1220.03,301170.00,2022,12,23,이더리움,1302.02,1588503.46
4,2022-12-22,1217.57,383630.00,2022,12,22,이더리움,1302.02,1585300.49


In [49]:
# 컬럼명 변경
eth_krw.rename(columns={'종가':'종가(USD)'}, inplace=True)
# 컬럼 순서 변경
eth_krw = eth_krw[['종류','날짜','연','월','일','종가(USD)','종가(KRW)','원달러환율', '거래량']]

In [50]:
eth_krw.head()
# eth_krw 데이터 전처리. 끝.

,종류,날짜,연,월,일,종가(USD),종가(KRW),원달러환율,거래량
0,이더리움,2022-12-26,2022,12,26,1218.49,1586498.35,1302.02,258530.00
1,이더리움,2022-12-25,2022,12,25,1218.69,1586758.75,1302.02,256520.00
2,이더리움,2022-12-24,2022,12,24,1220.46,1589063.33,1302.02,131510.00
3,이더리움,2022-12-23,2022,12,23,1220.03,1588503.46,1302.02,301170.00
4,이더리움,2022-12-22,2022,12,22,1217.57,1585300.49,1302.02,383630.00


### wemix_copy 데이터 전처리

In [51]:
# eth_copy에서 진행했던 절차를 따라 진행.
print(wemix_copy.isnull().sum()) # NaN 없음
print(wemix_copy.info()) # 데이터가 이미 float 이기 때문에 변환 절차 필요 없음.

날짜      0
종가      0
오픈      0
고가      0
저가      0
거래량     0
변동 %    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      210 non-null    object 
 1   종가      210 non-null    float64
 2   오픈      210 non-null    float64
 3   고가      210 non-null    float64
 4   저가      210 non-null    float64
 5   거래량     210 non-null    object 
 6   변동 %    210 non-null    object 
dtypes: float64(4), object(3)
memory usage: 11.6+ KB
None


In [52]:
# 오픈, 고가, 저가, 변동 % 삭제
wemix_copy.drop(wemix_copy.columns[[2,3,4,6]], axis=1, inplace=True)
wemix_copy.head()

,날짜,종가,거래량
0,2022년 12월 26일,0.37,715.09K
1,2022년 12월 25일,0.37,631.54K
2,2022년 12월 24일,0.37,1.34M
3,2022년 12월 23일,0.37,1.58M
4,2022년 12월 22일,0.37,1.68M


In [53]:
# 거래량에 K 뿐만이 아니라 M이 있는 것을 확인함.
wemix_copy['거래량'].unique()

array(['715.09K', '631.54K', '1.34M', '1.58M', '1.68M', '1.31M', '1.98M',
       '2.14M', '1.37M', '2.81M', '2.73M', '3.13M', '3.62M', '6.24M',
       '10.09M', '21.93M', '33.50M', '9.06M', '224.25M', '131.91M',
       '16.39M', '16.87M', '19.17M', '17.01M', '24.94M', '3.23M',
       '344.91K', '595.35K', '704.81K', '910.50K', '2.35M', '4.19M',
       '9.37M', '1.18M', '1.46M', '1.69M', '2.10M', '861.76K', '865.21K',
       '806.67K', '1.10M', '1.79M', '1.25M', '1.08M', '3.51M', '3.68M',
       '1.09M', '1.64M', '846.03K', '1.32M', '708.89K', '4.04M',
       '617.14K', '887.02K', '1.71M', '1.49M', '2.29M', '12.88M',
       '695.34K', '744.16K', '856.43K', '616.67K', '645.31K', '837.96K',
       '1.03M', '1.00M', '1.12M', '1.30M', '752.42K', '1.27M', '1.52M',
       '1.57M', '1.13M', '838.07K', '264.36K', '196.84K', '234.72K',
       '412.49K', '331.79K', '294.30K', '511.92K', '485.80K', '332.59K',
       '377.49K', '710.29K', '861.70K', '1.67M', '868.51K', '326.38K',
       '378.95K', 

In [54]:
# K인 경우와 M인 경우 따로 처리 진행
for i in range(len(wemix_copy['거래량'])):
    if wemix_copy['거래량'][i][-1] == 'K': # 마지막 단어가 K인 경우
        wemix_copy['거래량'][i] = wemix_copy['거래량'][i][:-1] # 마지막 단어 K를 없앰
        wemix_copy['거래량'][i] = float(wemix_copy['거래량'][i])
        wemix_copy['거래량'][i] *= 1000 # 단위수 맞춤
    elif wemix_copy['거래량'][i][-1] == 'M':
        wemix_copy['거래량'][i] = wemix_copy['거래량'][i][:-1] # 마지막 단어 M을 없앰
        wemix_copy['거래량'][i] = float(wemix_copy['거래량'][i])
        wemix_copy['거래량'][i] *= 1000000 # 단위수 맞춤

C:\Users\Jasper\AppData\Local\Temp\ipykernel_32912\615921124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wemix_copy['거래량'][i] = wemix_copy['거래량'][i][:-1] # 마지막 단어 K를 없앰
C:\Users\Jasper\AppData\Local\Temp\ipykernel_32912\615921124.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wemix_copy['거래량'][i] = float(wemix_copy['거래량'][i])
C:\Users\Jasper\AppData\Local\Temp\ipykernel_32912\615921124.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [55]:
wemix_copy.head()

,날짜,종가,거래량
0,2022년 12월 26일,0.37,715090.00
1,2022년 12월 25일,0.37,631540.00
2,2022년 12월 24일,0.37,1340000.00
3,2022년 12월 23일,0.37,1580000.00
4,2022년 12월 22일,0.37,1680000.00


In [56]:
# 날짜를 연, 월, 일로 분리하고 싶음

# 날짜의 데이터는 '2022년 12월 26일'로 구분자가 3개가 있음.
# 구분자를 하나로 변경.
wemix_copy['날짜'] = wemix_copy['날짜'].apply(lambda x: x.replace('년 ','-').replace('월 ', '-').replace('일', ''))

dates = wemix_copy['날짜'].str.split('-') # 리스트로 데이터 리턴
wemix_copy['연'] = dates.str.get(0)
wemix_copy['월'] = dates.str.get(1)
wemix_copy['일'] = dates.str.get(2)

wemix_copy.head()

,날짜,종가,거래량,연,월,일
0,2022-12-26,0.37,715090.00,2022,12,26
1,2022-12-25,0.37,631540.00,2022,12,25
2,2022-12-24,0.37,1340000.00,2022,12,24
3,2022-12-23,0.37,1580000.00,2022,12,23
4,2022-12-22,0.37,1680000.00,2022,12,22


In [57]:
# '종류'라는 컬럼에 '위믹스' 추가
wemix_copy['종류'] = '위믹스'
wemix_copy.head()

,날짜,종가,거래량,연,월,일,종류
0,2022-12-26,0.37,715090.00,2022,12,26,위믹스
1,2022-12-25,0.37,631540.00,2022,12,25,위믹스
2,2022-12-24,0.37,1340000.00,2022,12,24,위믹스
3,2022-12-23,0.37,1580000.00,2022,12,23,위믹스
4,2022-12-22,0.37,1680000.00,2022,12,22,위믹스


In [58]:
# 하다보니 현재 금액이 USD로 되어 있기 때문에 KRW로 변환하고 싶음
# 기 처리된 ex_rate 데이터프레임 사용

# wemix_krw df 새로 생성
wemix_krw = wemix_copy.copy()
# 원달러환율 컬럼 추가, ex_rate의 연월 일치하는 환율을 찾아서 대입
wemix_krw['원달러환율'] = 0

# 계산을 위한 초기 변수 설정
years = ['2020','2021','2022']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
krw_rate = 0

for year in years:
    for month in months:
        krw_rate = ex_rate.loc[(ex_rate['연'] ==year) & (ex_rate['월'] ==month),['환율']]
        krw_rate = krw_rate.squeeze() #데이터프레임을 시리즈로 변환하여 계산할수 있게 함.
        wemix_krw.loc[(wemix_krw['연'] ==year) & (wemix_krw['월'] ==month),['원달러환율']] = krw_rate
print(wemix_krw.head())
print(wemix_krw.tail())

           날짜   종가        거래량     연   월   일   종류   원달러환율
0  2022-12-26 0.37  715090.00  2022  12  26  위믹스 1302.02
1  2022-12-25 0.37  631540.00  2022  12  25  위믹스 1302.02
2  2022-12-24 0.37 1340000.00  2022  12  24  위믹스 1302.02
3  2022-12-23 0.37 1580000.00  2022  12  23  위믹스 1302.02
4  2022-12-22 0.37 1680000.00  2022  12  22  위믹스 1302.02
             날짜   종가       거래량     연   월   일   종류   원달러환율
205  2022-06-04 3.78 118520.00  2022  06  04  위믹스 1277.35
206  2022-06-03 3.56  88030.00  2022  06  03  위믹스 1277.35
207  2022-06-02 3.88 159640.00  2022  06  02  위믹스 1277.35
208  2022-06-01 3.91 227160.00  2022  06  01  위믹스 1277.35
209  2022-05-31 4.32 182420.00  2022  05  31  위믹스 1269.88


In [59]:
# 종가(KRW) 컬럼 추가
wemix_krw['종가(KRW)'] = wemix_krw['종가'] * wemix_krw['원달러환율']
wemix_krw.head()

,날짜,종가,거래량,연,월,일,종류,원달러환율,종가(KRW)
0,2022-12-26,0.37,715090.00,2022,12,26,위믹스,1302.02,483.44
1,2022-12-25,0.37,631540.00,2022,12,25,위믹스,1302.02,479.14
2,2022-12-24,0.37,1340000.00,2022,12,24,위믹스,1302.02,481.36
3,2022-12-23,0.37,1580000.00,2022,12,23,위믹스,1302.02,482.79
4,2022-12-22,0.37,1680000.00,2022,12,22,위믹스,1302.02,478.10


In [60]:
# 컬럼명 변경
wemix_krw.rename(columns={'종가':'종가(USD)'}, inplace=True)
# 컬럼 순서 변경
wemix_krw = wemix_krw[['종류','날짜','연','월','일','종가(USD)','종가(KRW)','원달러환율', '거래량']]

In [61]:
wemix_krw.head()
# wemix_krw 데이터 전처리. 끝.

,종류,날짜,연,월,일,종가(USD),종가(KRW),원달러환율,거래량
0,위믹스,2022-12-26,2022,12,26,0.37,483.44,1302.02,715090.00
1,위믹스,2022-12-25,2022,12,25,0.37,479.14,1302.02,631540.00
2,위믹스,2022-12-24,2022,12,24,0.37,481.36,1302.02,1340000.00
3,위믹스,2022-12-23,2022,12,23,0.37,482.79,1302.02,1580000.00
4,위믹스,2022-12-22,2022,12,22,0.37,478.10,1302.02,1680000.00


### 통합 데이터 프레임(concat) 생성 및 인덱싱 진행

In [72]:
# bit_krw, eth_krw, wemix_krw concat 진행
total_df = pd.concat([bit_krw,eth_krw,wemix_krw],ignore_index=True)

In [73]:
# 인덱싱은 '종류' 컬럼으로 진행
total_df = total_df.set_index('종류')
total_df

,날짜,연,월,일,종가(USD),종가(KRW),원달러환율,거래량
종류,,,,,,,,
비트코인,2022-12-26,2022,12,26,16831.90,21915470.44,1302.02,133330.00
비트코인,2022-12-25,2022,12,25,16831.80,21915340.24,1302.02,132890.00
비트코인,2022-12-24,2022,12,24,16837.20,21922371.14,1302.02,110120.00
비트코인,2022-12-23,2022,12,23,16779.10,21846723.78,1302.02,184120.00
비트코인,2022-12-22,2022,12,22,16820.60,21900757.61,1302.02,198280.00
...,...,...,...,...,...,...,...,...
위믹스,2022-06-04,2022,06,04,3.78,4829.28,1277.35,118520.00
위믹스,2022-06-03,2022,06,03,3.56,4551.73,1277.35,88030.00
위믹스,2022-06-02,2022,06,02,3.88,4954.87,1277.35,159640.00
